## Lab #4 Assignment
## QMSS S5019 - Data Analysis with Python
## Lovina Putri
## CUID: lap2236

***(1) Set-up packages and code***

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas_datareader as web
import datetime

***(2) Import data***

I will use the US' macroeconomic indicators such as Real Gross Domestic Product (real GDP) and Real Gross Private Domestic Investment (real GFCF -- real Gross Fixed Capital Formation, which describes private sector investments) from the FRED St. Louis database.

Description of variables:
1.   **Real GDP**: Real gross domestic product is the inflation adjusted value of the goods and services produced by labor and property located in the United States. **Units**: Billions of Chained 2017 Dollars, Seasonally Adjusted Annual Rate. **Frequency**: Quarterly.
2.  **Real GFCF**: Real private fixed investment (PFI) measures spending by private businesses, nonprofit institutions, and households on fixed assets in the U.S. economy ajusted by the inflation. **Units**: Billions of Chained 2017 Dollars, Seasonally Adjusted Annual Rate. **Frequency**: Quarterly.  



In [ ]:
today = pd.to_datetime("today")
start = datetime.datetime(1960, 1, 1)
end = today

In [ ]:
df_GDP = web.DataReader(['GDPC1'], 'fred', start, end)
df_GPDI = web.DataReader(['GPDIC1'], 'fred', start, end)

***(3) Data check***

In [ ]:
df_GDP.head()

,GDPC1
DATE,
1960-01-01,3517.181
1960-04-01,3498.246
1960-07-01,3515.385
1960-10-01,3470.278
1961-01-01,3493.703


In [ ]:
df_GPDI.head()

,GPDIC1
DATE,
1960-01-01,406.581
1960-04-01,368.686
1960-07-01,367.749
1960-10-01,327.010
1961-01-01,335.496


***(4) Data Management***

1. Merge Data

In [ ]:
df = pd.merge(df_GDP, df_GPDI, left_index=True, right_index=True)
df.index = df.index + pd.offsets.QuarterEnd(0)
print(df)

                GDPC1    GPDIC1
DATE                           
1960-03-31   3517.181   406.581
1960-06-30   3498.246   368.686
1960-09-30   3515.385   367.749
1960-12-31   3470.278   327.010
1961-03-31   3493.703   335.496
...               ...       ...
2024-03-31  23053.545  4282.515
2024-06-30  23223.906  4369.185
2024-09-30  23400.294  4377.736
2024-12-31  23542.349  4315.094
2025-03-31  23512.717  4551.994

[261 rows x 2 columns]


2. Rename Variables

In [ ]:
df.rename(columns={
    'GDPC1': 'real_gdp',
    'GPDIC1': 'real_gfcf'
}, inplace=True)

In [ ]:
df.columns

Index(['real_gdp', 'real_gfcf'], dtype='object')

3. Create log-transformed variables

In [ ]:
df['ln_real_gdp'] = np.log(df['real_gdp'])
df['ln_real_gfcf'] = np.log(df['real_gfcf'])

4. Describe the dataset

In [ ]:
df.describe()

,real_gdp,real_gfcf,ln_real_gdp,ln_real_gfcf
count,261.000000,261.000000,261.000000,261.000000
mean,11662.289820,1803.995161,9.225505,7.267990
std,5808.049054,1156.328057,0.546730,0.707825
min,3470.278000,327.010000,8.151990,5.789991
25%,6370.025000,800.370000,8.759359,6.685074
50%,10449.673000,1326.029000,9.254326,7.189944
75%,16713.314000,2692.179000,9.723961,7.898106
max,23542.349000,4551.994000,10.066556,8.423321


5. Create variable of simple time trend for each quarter

In [ ]:
df['time_trend'] = np.arange(len(df))
df.head()

,real_gdp,real_gfcf,ln_real_gdp,ln_real_gfcf,time_trend
DATE,,,,,
1960-03-31,3517.181,406.581,8.165415,6.007783,0
1960-06-30,3498.246,368.686,8.160017,5.909945,1
1960-09-30,3515.385,367.749,8.164904,5.907401,2
1960-12-31,3470.278,327.010,8.151990,5.789991,3
1961-03-31,3493.703,335.496,8.158717,5.815610,4


***(5) Questions***

**1 -- Run a naive OLS regression on your time series data. Tell me how you expect your Xs to affect your Y and why.   Interpret your results.**

I expect that the rise in real GDP will increase real GFCF since investment responds to the output or GDP growth.

In [ ]:
# Regression 1 (Levels, OLS):
model1 = smf.ols('ln_real_gfcf ~ ln_real_gdp', data=df).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:           ln_real_gfcf   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                 2.091e+04
Date:                Fri, 27 Jun 2025   Prob (F-statistic):          1.09e-249
Time:                        03:57:57   Log-Likelihood:                 295.01
No. Observations:                 261   AIC:                            -586.0
Df Residuals:                     259   BIC:                            -578.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -4.6025      0.082    -55.971      

From the OLS result above, a 1% increase in real GDP growth is associated with a 1.29% of real GFCF growth on average, ceteris paribus. The log of real GDP variable allow us to predict log(GFCF) with 98.8% more accuracy, compared to just guessing the average of log(GFCF).

However, this could be a spurious regression and we should check the time-series component of these variables to see the relationship in the short or long-run. The result also describe that 98.8% of the variances of the log(GFCF) is explained only by log(GDP), while in reality other factors could also affect GFCF significantly.

**2 -- Run a first differences regression on the same model in Question 1. Interpret your results. Do you draw a different conclusion than in Question 1? Explain.**

In [ ]:
# Regression 2: Using first differences

# Apply diff() only to the numeric columns and drop any resulting NaN rows
# Sort the DataFrame by the DATE index to ensure chronological order
df = df.sort_index()

# Apply diff() to calculate differences for GDP and GFCF
df_diff = df[['ln_real_gdp', 'ln_real_gfcf']].diff().dropna()

model2 = smf.ols(formula = 'ln_real_gfcf ~ ln_real_gdp', data = df_diff).fit()
print (model2.summary())

                            OLS Regression Results                            
Dep. Variable:           ln_real_gfcf   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.606
Method:                 Least Squares   F-statistic:                     399.6
Date:                Fri, 27 Jun 2025   Prob (F-statistic):           2.43e-54
Time:                        03:57:57   Log-Likelihood:                 586.12
No. Observations:                 260   AIC:                            -1168.
Df Residuals:                     258   BIC:                            -1161.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0125      0.002     -6.511      

Using the first-difference of log variables, we can see that a 1% increase in log(GDP) (real GDP growth) will raise the log(GFCF) (real GFCF growth) by 2.98% on average, ceteris paribus. Note that the coefficient is larger in magnitude when compared to the percentage level model in OLS regression since we are looking at the changes in GFCF from one period to other, rather than the overall levels of GFCF.

We can also observe that the Adj. R-Squared is falling from 0.988 in levels model to 0.606 -- which indicates that this model reflects only 60.6% of the variance -- which is 30.2 percentage points lower than the OLS model. This reflects that short-term fluctuations are more difficult to explain than long-term levels. However, this differenced-log specification avoids issues of non-stationarity and gives a clearer view of short-run elasticities.

**Extras:**

1. Add quarter dummies to account for seasonality:

In [ ]:
# Extract the quarter number from the index and create a new column
df['quarter_num'] = df.index.quarter

In [ ]:
# Extract the quarter name from the index and create a new column
df['quarter_name'] = df.index.strftime('%B')

In [ ]:
# Sort the DataFrame by the quarter index to ensure time order is correct
df = df.sort_index()

# Compute first differences for GDP and GFCF
df['ln_real_gdp_diff'] = df['ln_real_gdp'].diff()
df['ln_real_gfcf_diff'] = df['ln_real_gfcf'].diff()

# Create a column for quarter-to-quarter transitions with correct order
df['quarter_transition'] = df['quarter_name'] + " to " + df['quarter_name'].shift(-1)

# Drop the last row where the transition would be NaN due to the shift
df.dropna(inplace=True)

# Display the updated DataFrame
print(df)

             real_gdp  real_gfcf  ln_real_gdp  ln_real_gfcf  time_trend  \
DATE                                                                      
1960-06-30   3498.246    368.686     8.160017      5.909945           1   
1960-09-30   3515.385    367.749     8.164904      5.907401           2   
1960-12-31   3470.278    327.010     8.151990      5.789991           3   
1961-03-31   3493.703    335.496     8.158717      5.815610           4   
1961-06-30   3553.021    359.004     8.175554      5.883334           5   
...               ...        ...          ...           ...         ...   
2023-12-31  22960.600   4244.835    10.041535      8.353458         255   
2024-03-31  23053.545   4282.515    10.045575      8.362296         256   
2024-06-30  23223.906   4369.185    10.052937      8.382332         257   
2024-09-30  23400.294   4377.736    10.060504      8.384287         258   
2024-12-31  23542.349   4315.094    10.066556      8.369874         259   

            quarter_num 

In [ ]:
df.quarter_transition.value_counts(normalize=True).sort_index()*100

,proportion
quarter_transition,
December to March,25.096525
June to September,25.096525
March to June,24.710425
September to December,25.096525


2. Transition from one quarter to another in terms of GDP and GFCF

In [ ]:
# Define the logical order of the quarter transitions
ordered_quarter = ["December to March", "March to June", "June to September", "September to December"]

# Convert 'quarter_transition' into an ordered categorical type
df['ordered_quarter'] = pd.Categorical(df['quarter_transition'], categories=ordered_quarter, ordered=True)

# Sort the DataFrame based on the new categorical order
df = df.sort_values('quarter_transition')

# Display the DataFrame
print(df)

             real_gdp  real_gfcf  ln_real_gdp  ln_real_gfcf  time_trend  \
DATE                                                                      
2024-12-31  23542.349   4315.094    10.066556      8.369874         259   
1999-12-31  13827.980   2367.988     9.534449      7.769796         159   
1976-12-31   6451.177    771.471     8.772018      6.648299          67   
2000-12-31  14229.765   2471.534     9.563091      7.812594         163   
1975-12-31   6184.530    669.438     8.729806      6.506439          63   
...               ...        ...          ...           ...         ...   
1975-09-30   6102.326    651.260     8.716425      6.478909          62   
2001-09-30  14214.516   2311.409     9.562019      7.745613         166   
2000-09-30  14145.312   2466.375     9.557139      7.810505         162   
1999-09-30  13604.771   2306.470     9.518176      7.743473         158   
1992-09-30  10449.673   1289.101     9.254326      7.161700         130   

            quarter_num 

3. Additional regressions

In [ ]:
# Regression 3: First difference + quarter transition + time trend

model3 = smf.ols(formula = 'ln_real_gfcf_diff ~ ln_real_gdp_diff + quarter_transition + time_trend', data = df).fit()
print (model3.summary())

                            OLS Regression Results                            
Dep. Variable:      ln_real_gfcf_diff   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     84.32
Date:                Fri, 27 Jun 2025   Prob (F-statistic):           7.13e-52
Time:                        03:57:57   Log-Likelihood:                 589.80
No. Observations:                 259   AIC:                            -1168.
Df Residuals:                     253   BIC:                            -1146.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

Net of quarter-to-quarter transitions and time trend, for each 1% increase in real GDP growth lead to an increase of real GFCF growth by 3.052%, on average and statistically significant at 5% level. The Adj. R-Squared slightly increased to 61.8% compared to previous model without time trend and quarterly transitions.

In [ ]:
# Regression 4: Levels + a linear time trend

# Sort the DataFrame by the Quarter index to ensure time order is correct
df = df.sort_index()
df['time_trend'] = np.arange(len(df))

model4 = smf.ols(formula = 'ln_real_gfcf ~ ln_real_gdp + time_trend', data = df).fit()
print (model4.summary())

                            OLS Regression Results                            
Dep. Variable:           ln_real_gfcf   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                 1.029e+04
Date:                Fri, 27 Jun 2025   Prob (F-statistic):          2.91e-245
Time:                        03:57:57   Log-Likelihood:                 294.10
No. Observations:                 259   AIC:                            -582.2
Df Residuals:                     256   BIC:                            -571.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -5.5500      0.675     -8.223      

Net of time trend, for each 1% increase in real GDP growth lead to an increase of real GFCF growth by 1.4%, on average and statistically significant at 5% level. The Adj. R-Squared increased to 98.7% compared to model 3 at 61%.

In [ ]:
# Regression 5: Levels + seasonality

model5 = smf.ols(formula = 'ln_real_gfcf ~ ln_real_gdp + C(quarter_num) + time_trend', data = df).fit()
print (model5.summary())

                            OLS Regression Results                            
Dep. Variable:           ln_real_gfcf   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     4067.
Date:                Fri, 27 Jun 2025   Prob (F-statistic):          2.25e-239
Time:                        03:57:57   Log-Likelihood:                 294.11
No. Observations:                 259   AIC:                            -576.2
Df Residuals:                     253   BIC:                            -554.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -5.5507    

Net of categorical quarter (seasonality) and time trend, for each 1% increase in real GDP growth lead to an increase of real GFCF growth by 1.4%, on average and statistically significant at 5% level. The Adj. R-Squared increased to 98.7%.

4. Cointegration test

In [ ]:
from statsmodels.tsa.stattools import coint

# Perform the cointegration test
coint_result = coint(df['ln_real_gdp'], df['ln_real_gfcf'])

# Extract the test statistic, p-value, and critical values
test_statistic = coint_result[0]
p_value = coint_result[1]
critical_values = coint_result[2]

# Display the results
print("Test Statistic:", test_statistic)
print("p-value:", p_value)
print("Critical Values:", critical_values)

Test Statistic: -3.700884734734846
p-value: 0.01827746694968465
Critical Values: [-3.93939291 -3.35991506 -3.06092962]


* Test Statistic vs. Critical Values:  The test statistic of -3.7 is lower than 5% critical value (-3.9394, -3.3599, and -3.0609, at the 1%, 5%, and 10% levels respectively). This indicates that the test statistic not fall in the rejection region of 5% significance level, meaning we can reject the null hypothesis of no cointegration and this indicates that there is a long-run cointegration relationship between ln_real_gdp and ln_real_gfcf.

* p-value:  The p-value of 0.01827 is lower than 0.05 which implies that we can reject the null hypothesis of no cointegration.

* Conclusion:  The results suggest that there is an evidence of cointegration between the real GDP growth and real GFCF growth. In other words, the two series do share a long-term equilibrium relationship. They move together in the short- and long-term.

---